In [ ]:
import json
import torch
import custom_utils
import numpy as np

import optuna

torch.set_default_device('cpu')

In [ ]:
# importing data
with open('data/X_train.json', 'r') as json_file:
    X_train = json.load(json_file)
   
with open('data/y_train.json', 'r') as json_file:
    y_train = json.load(json_file)
   
with open('data/X_test.json', 'r') as json_file:
    X_test = json.load(json_file) 

with open('data/y_test.json', 'r') as json_file:
    y_test = json.load(json_file)
    
# converting to appropriate format (and device)
X_train = torch.Tensor(X_train).float()
y_train = torch.Tensor(y_train).long()

X_test = torch.Tensor(X_test).float()
y_test = torch.Tensor(y_test).long()

# getting shapes
num_features = X_train.shape[1]
num_classes = 2
# edge_dim = edge_attr.shape[1]

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from torchmetrics.classification import F1Score

In [ ]:
def MLP(trial):
    # Model
    n_layers = trial.suggest_int('n_layers', 1, 4)
    layers = []

    in_features = num_features
    for i in range(n_layers):
        out_features = trial.suggest_int(f'n_units_l{i}', 200, 800)
        layers.append(torch.nn.Linear(in_features, out_features))
        layers.append(torch.nn.ReLU())
        
        # suggest dropout
        p = trial.suggest_float(f'p{i}', 0, 0.8)
        layers.append(torch.nn.Dropout(p))

        # updating next layer size
        in_features = out_features
        
    layers.append(torch.nn.Linear(in_features, num_classes))
    model = torch.nn.Sequential(*layers)
    return model

In [ ]:
# set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def objective(trial):
    
    def train(X_train, y_train):
        model.train() 
        optimizer.zero_grad()
        out = model(X_train)
        loss = criterion(out, y_train)
        loss.backward()
        optimizer.step()
        return loss
    
    def validate(X_valid, y_valid):
        model.eval()
        out = model(X_valid)
        loss = criterion(out, y_valid)
        y_pred = out.argmax(dim=1)
        f1 = F1Score(task='binary', num_classes=num_classes).to(device)
        score = f1(y_valid, y_pred)
        # score = f1_score(y_valid, y_pred)
        return loss, score
        
    
    n_folds = 5
    n_epochs = 200
    patience = 10
    avg_score = 0
    skf = StratifiedKFold(n_splits=n_folds)
    for i, (train_idx, val_idx) in enumerate(skf.split(X_train,y_train)):
        
        # set model, criterion and optimizers
        model = MLP(trial).to(device)
        criterion = torch.nn.CrossEntropyLoss() # need to check input
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

        # Selecing fold train and validation
        train_mask = torch.zeros(len(X_train), dtype=torch.bool)
        val_mask = torch.zeros(len(X_train), dtype=torch.bool)
        train_mask[train_idx] = True
        val_mask[val_idx] = True
        
        X_train_fold = X_train[train_mask].to(device)
        y_train_fold = y_train[train_mask].to(device)
        X_valid_fold = X_train[val_mask].to(device)
        y_valid_fold = y_train[val_mask].to(device)
        
        # Epoch
        best_valid_loss = float('inf')
        score_at_best = -1
        current_patience = 0
        for epoch in range(n_epochs):
            train_loss = train(X_train_fold, y_train_fold)
            valid_loss, score = validate(X_valid_fold, y_valid_fold)
           
            # Stopping criterium            
            if valid_loss > best_valid_loss:
                current_patience += 1
            else:
                best_valid_loss = valid_loss
                score_at_best = score 
                current_patience = 0
            
            if current_patience == patience:
                break
            
            print(f'Epoch: {epoch}, Train loss: {train_loss:.4f}, Valid loss: {valid_loss:.4f}, Score: {score:.4f}')
        
        avg_score += score_at_best/n_folds
        
    return avg_score

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)